In [1]:
from naics_gemini.data_loader.datamodule import NAICSDataModule

In [2]:
streaming_cfg = {
    'anchor_level': [2],
    'n_positives': 1,
    'n_negatives': 5
}

In [3]:
DataModule = NAICSDataModule(
    streaming_config=streaming_cfg
)

In [4]:
train_loader = DataModule.train_dataloader()

In [5]:
batch = next(iter(train_loader))

In [6]:
batch

{'anchor': {'title': {'input_ids': tensor([[  101,  5237,  1010, 13116,  1010,  5645,  1998,  5933,   102,     0,
                0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
                0,     0,     0,     0],
           [  101,  5471,  1010, 12907,  2075,  1010,  1998,  3514,  1998,  3806,
            14676,   102,     0,     0,     0,     0,     0,     0,     0,     0,
                0,     0,     0,     0],
           [  101, 16548,   102,     0,     0,     0,     0,     0,     0,     0,
                0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
                0,     0,     0,     0],
           [  101,  2810,   102,     0,     0,     0,     0,     0,     0,     0,
                0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
                0,     0,     0,     0],
           [  101,  5814,   102,     0,     0,     0,     0,     0,     0,     0,
                0,     0,     0,     0,     0,     0,     0,     0